In [1]:
import sys
from transformers import pipeline
from sklearn.metrics import f1_score
import pandas as pd
from datasets import  load_dataset


/home/veton/anaconda3/envs/bfh/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sys.path.append('../code/')

In [9]:
from training_data_handler import TrainingDataHandler
tdh = TrainingDataHandler()

In [10]:
#tdh = TrainingDataHandler()
#classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")

classifier = pipeline("zero-shot-classification", model="Sahajtomar/German_Zeroshot")

In [22]:
language = 'de'
tdh.get_training_data(language=language, affair_text_scope=['zh','ch'], text=True, title=True)
test_set_only_titles = tdh.training_data_df[tdh.training_data_df.split=="test"]
docs = test_set_only_titles[test_set_only_titles.text.apply(lambda x: len(x)<512)]
docs = docs.reset_index()

In [20]:
def classify(sequence_to_classify):
    
    output = classifier(sequence_to_classify, list(tdh.label2id.keys()), multi_label=True)
    labels_scores = list(zip(output["labels"], output["scores"]))
    labels_scores = [res[0] for res in labels_scores if res[1]>0.5]
    return labels_scores
    


In [28]:
for index in range(0,100):
    # index = 0
    affair_text_srcid= docs.affair_text_srcid.to_list()[index]
    text = docs.text.to_list()[index]
    labels = docs.affair_topic_codes_as_labels.to_list()[index]
    output = classify(text)
    if any([x for x in output if x in labels])==False:
        print(affair_text_srcid)
        print(text)
        print(labels)

        print(output)
        print('\n################################################\n')

20095283
Wie beurteilt der Bundesrat die neueste Idee der Sterbehilfeorganisationen, auch Gesunden Sterbehilfe leisten zu wollen?
['Soziale Frage']
['Strategien im Gesundheitssystem', 'Gesundheit', 'Medizinische Versorgung']

################################################

20095327
Die Aargauer Behörden haben den Fall Lucie untersucht. Für ihren Tod übernimmt - wie in allen ähnlichen Fällen - niemand die Verantwortung. Der Bericht zeigt auf, dass das System versagt hat. Der Kanton Aargau hat nun entsprechende Massnahmen getroffen.- Was hat der Bundesrat in dieser Sache seit März getan?- Welchen Handlungsbedarf sieht er auf nationaler Ebene (vgl. Antwort des Bundesrates auf die Frage 09.5090)?
['Recht allgemein']
['Politisches System', 'Sozialer Schutz', 'Strategien im Gesundheitssystem', 'Forschung', 'Medizinische Versorgung', 'Soziale Frage']

################################################

20095366
Der SRG-Generaldirektor tritt per Ende 2010 zurück. Die SRG hat ein Personal- und 

20105222
Im vergangenen Jahr stieg im Tessin parallel zur Arbeitslosigkeit die Zahl der Grenzgängerinnen und Grenzgänger. In verschiedenen Branchen könnte dies auf einen Ersatz der Arbeitskräfte hindeuten.Ist es für den Bundesrat denkbar, die Beschäftigungspolitik der Kantone, die einem solchen Druck ausgesetzt sind, stärker zu unterstützen?
['Wirtschaft']
['Soziale Frage', 'Beschäftigung und Arbeit', 'Forschung', 'Verkehr', 'Sozialer Schutz', 'Migration und Asyl']

################################################

20105252
Das Sekretariat der Weko hat Ende Mai 2010 Bussen gegen Firmen des Luxus-Parfümerie- und Kosmetikmarktes beantragt. Es geht um den "Austausch von Marktinformationen, die gegenseitige Anpassung von Preisen und das Einfrieren von Marktanteilen". Laut vorliegenden Indizien könnten solche Sachverhalte auch auf die Hörgerätebranche zutreffen, gegen welche die Weko Anfang Jahr eine Voruntersuchung eröffnet haben soll.Wann mündet diese Voruntersuchung in eine Untersuchung 

20115296
Was unternimmt der Bundesrat, um die auch 20 Jahre nach dem Frauenstreik noch immer bestehende Lohndiskriminierung der Frauen beim Staat und in der Privatwirtschaft umgehend zu beseitigen?
['Wirtschaft']
['Soziale Frage', 'Forschung', 'Sozialer Schutz', 'Beschäftigung und Arbeit', 'Sicherheit', 'Alter', 'Politisches System', 'Pflege']

################################################

20115297
- Ist der Bundesrat bereit, einen landesweiten Frauenstreik am Arbeitsplatz und im Bereich der privaten Betreuungs- und Pflegearbeit zu unterstützen, um die Gleichstellung der Geschlechter zu beschleunigen?- Oder ergreift er andere Massnahmen, um dieses Ziel zu erreichen? Wenn ja, welche? Die Antwort des Bundesrates existiert nur in französischer Sprache.
['Wirtschaft']
['Soziale Frage', 'Forschung', 'Beschäftigung und Arbeit', 'Pflege', 'Sozialer Schutz', 'Strategien im Gesundheitssystem']

################################################

20115319
Arbeitgeber, Arbeitnehmer und Bund hab